In [1]:
import operator
from typing import Annotated, TypedDict
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, END
from IPython.display import display, Markdown

In [2]:
# --- 1. Define the Graph State ---
# - This defines the "schema" of the state that will be passed between nodes.

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The user's initial question.
        response: The accumulated response from the LLMs.
    """
    question: str
    response: Annotated[str, operator.add] # Use operator.add to append responses

In [3]:
# --- 2. Initialize Local LLMs ---
# Assuming you have Ollama running and these models downloaded:
# ollama run llama3:8b

try:
    llm_1 = ChatOllama(model="qwen3:0.6b", temperature=0) # Often a good general-purpose lightweight model
    llm_2 = ChatOllama(model="qwen3:0.6b", temperature=0) # Another popular lightweight choice
    llm_3 = ChatOllama(model="qwen3:0.6b", temperature=0) # Very small, good for basic tasks
except Exception as e:
    print(f"Error initializing Ollama models. Make sure Ollama is running and models are downloaded.")
    print(f"You can download models with: ollama run qwen3:0.6b")
    print(f"Error: {e}")
    exit()

In [4]:
# --- 3. Define Nodes (Functions for each LLM) ---

def llm_node_1(state: GraphState):
    """
    First LLM processing node.
    """
    print("---NODE 1: Qwen Processing---")
    question = state["question"]
    # Simulate some processing by the LLM
    llm_response = llm_1.invoke(f"""Process this: {question}. Add a brief introduction. 
                                Start your final response with text **'Response 1:'** """)
    
    return {"response": f"Node 1 Qwen's take: {llm_response.content}\n"}

def llm_node_2(state: GraphState):
    """
    Second LLM processing node.
    """
    print("---NODE 2: Qwen Processing---")
    question = state["question"]
    current_response = state.get("response", "")
    #current_response = current_response.split("Response 1:")[-1]
    # Simulate more processing based on the question and previous response
    llm_response = llm_2.invoke(f"""Elaborate on: '{question}'. Consider previous output: '{current_response}' 
                                Start your final response with text **'Response 2:'** """)

    return {"response": f"Node 2 Qwen's take: {llm_response.content}\n"}


def llm_node_3(state: GraphState):
    """
    Third LLM processing node.
    """
    print("---NODE 3: Qwen Processing---")
    question = state["question"]
    current_response = state.get("response", "")
    # Simulate a final touch or summary
    llm_response = llm_3.invoke(f"""Summarize the main point of '{question}' after seeing this: '{current_response}'.
                                Start your final response with text **'Response 3:'** """)

    return {"response": f"Node 3 Qwen's take: {llm_response.content}\n"}

In [5]:
# --- 4. Build the LangGraph Workflow ---

workflow = StateGraph(GraphState)

# Add nodes to the workflow
workflow.add_node("QwenNode1", llm_node_1)
workflow.add_node("QwenNode2", llm_node_2)
workflow.add_node("QwenNode3", llm_node_3)

# Define the sequence of execution
workflow.set_entry_point("QwenNode1")
workflow.add_edge("QwenNode1", "QwenNode2")
workflow.add_edge("QwenNode2", "QwenNode3") 
workflow.add_edge("QwenNode3", END) 

# Compile the graph
app = workflow.compile()

In [6]:
user_question = "Explain the concept of photosynthesis in simple language."
for s in app.stream({"question": user_question, "response": ""}):
    node_name = list(s.keys())[0]
    display(Markdown(s[node_name]["response"]))
    print("="*25)

---NODE 1: Qwen Processing---


Node 1 Qwen's take: <think>
Okay, the user wants me to explain photosynthesis in simple language, add a brief introduction, and start with "Response 1:." Let me break this down.

First, photosynthesis is a process that plants use to make food. I need to explain it in simple terms. Maybe start with the basic idea. Then add a brief intro. Let me check if I'm using simple language. Words like "plants," "sunlight," "carbon dioxide," "oxygen" come to mind. Make sure to mention that plants make oxygen. Then wrap it up with the introduction. Let me structure it step by step. Avoid jargon, keep it conversational. Double-check the flow and ensure the introduction is brief. Alright, that should cover it.
</think>

**Response 1:**  
Photosynthesis is when plants use sunlight, water, and carbon dioxide to make oxygen and food. This process helps plants grow and supports life on Earth.  

**Explanation:**  
Plants take in sunlight, water, and carbon dioxide from the air and use them to create glucose and oxygen. This glucose is used by plants to build their own food, which then supports other plants and animals.


---NODE 2: Qwen Processing---


Node 2 Qwen's take: <think>
Okay, the user wants me to elaborate on the concept of photosynthesis in simple language. They also provided a previous response, so I need to build on that. Let me start by rephrasing the previous response to make it more detailed. The original response was a bit short, so expanding it would help. I should mention the process in more depth, maybe include some examples like how plants use the process to sustain life. Also, make sure to highlight the key components: sunlight, water, carbon dioxide, and the products. Keep the language simple and conversational. Let me check if I'm using simple terms and if the explanation is clear. Alright, that should cover it.
</think>

**Response 2:**  
Photosynthesis is when plants use sunlight, water, and carbon dioxide to make oxygen and food. This process helps plants grow and supports life on Earth.  

**Explanation:**  
Plants take in sunlight, water, and carbon dioxide from the air and use them to create glucose and oxygen. This glucose is used by plants to build their food, which then supports other plants and animals.  

**Key components:**  
- **Sunlight:** The energy needed for photosynthesis.  
- **Water:** The plant needs to absorb water to make food.  
- **Carbon dioxide:** The plant uses to create glucose.  
- **Products:** Oxygen is released back into the atmosphere, and plants make glucose for their own use.  

This process is vital for sustaining life on Earth.


---NODE 3: Qwen Processing---


Node 3 Qwen's take: <think>
Okay, the user wants me to summarize the main point of explaining photosynthesis in simple language. They provided a previous response, so I need to build on that. Let me check the previous response again to make sure I'm consistent.

In the first response, I mentioned plants using sunlight, water, and carbon dioxide to make oxygen and food. Then I expanded it by adding more details. The second response was a bit shorter, so I should expand on that. Let me rephrase the second response to be more detailed and include the key components. Also, make sure to mention the products and their purpose. Alright, that should cover it. Let me put it all together clearly.
</think>

**Response 3:**  
Photosynthesis is when plants use sunlight, water, and carbon dioxide to make oxygen and food. This process helps plants grow and supports life on Earth.  

**Explanation:**  
Plants take in sunlight, water, and carbon dioxide from the air and use them to create glucose and oxygen. This glucose is used by plants to build their food, which then supports other plants and animals.  

**Key components:**  
- **Sunlight:** The energy needed for photosynthesis.  
- **Water:** The plant needs to absorb water to make food.  
- **Carbon dioxide:** The plant uses to create glucose.  
- **Products:** Oxygen is released back into the atmosphere, and plants make glucose for their own use.  

This process is vital for sustaining life on Earth.
